# Bankomaty do wyrzucenia

In [1]:
numbers_to_remove = [79, 80, 81]

# Import RF & XGB

In [2]:
import pandas as pd
import glob
from utils import ends_with_number

# Define the folder where the CSV files are located
folder_path = r'D:\AGH\bankomaty_2022\data\redatyprognoz'

# Use glob to find all CSV files in the folder
file_pattern = folder_path + r'\prognozy_W_*.csv'
file_list = glob.glob(file_pattern)

# Create an empty list to hold the DataFrames
df_list = []

# Loop over each file and read it into a DataFrame
for file in file_list:
    df = pd.read_csv(file)
    period_id = df['date'].min()
    df['periodID'] = period_id
    df_list.append(df)

# Concatenate the DataFrames into a single DataFrame
xgb_raw_df = pd.concat(df_list)

# Print the resulting data frame
xgb_raw_df


,date,ATM_W_01,pred_W_01,ATM_W_02,pred_W_02,ATM_W_03,pred_W_03,ATM_W_04,pred_W_04,ATM_W_05,...,pred_W_77,ATM_W_78,pred_W_78,ATM_W_79,pred_W_79,ATM_W_80,pred_W_80,ATM_W_81,pred_W_81,periodID
0,2019-04-26,0.00000,18025.890447,36730.000000,32312.739975,24140.000000,20515.344047,53250.000000,39916.931243,55470.000000,...,13567.135704,21070.000000,23625.405025,5030.000000,3743.950777,14040.000000,10297.942157,50940.000000,33421.064130,2019-04-26
1,2019-04-27,0.00000,5496.196782,28830.000000,20792.240200,13960.000000,14268.481549,75880.000000,29137.418333,20020.000000,...,12397.889366,11790.000000,13060.321537,4700.000000,2487.449379,14810.000000,7609.871005,65700.000000,38782.425250,2019-04-26
2,2019-04-28,0.00000,966.456457,9740.000000,13136.783340,4810.000000,5666.156345,0.000000,707.341392,10570.000000,...,3018.251750,11930.000000,4137.395372,790.000000,1903.793577,6110.000000,5111.667196,12430.000000,6917.041783,2019-04-26
3,2019-04-29,0.00000,9184.834606,56280.000000,32412.737677,22570.000000,19141.433497,32950.000000,26186.652256,39740.000000,...,11510.732248,34590.000000,23353.389867,2290.000000,3990.237171,28830.000000,10765.845100,39220.000000,34538.109313,2019-04-26
4,2019-04-30,0.00000,5421.585490,63850.000000,45630.105354,27850.000000,26586.427706,62430.000000,40676.321181,38090.000000,...,22350.662557,30310.000000,32451.121180,3980.000000,3026.529930,20320.000000,22086.625174,52650.000000,42971.444467,2019-04-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,2021-01-29,14760.00000,20151.204157,28910.000000,39241.788153,36180.000000,15584.018858,25520.000000,36132.547857,34020.000000,...,14032.875367,720.000000,4713.536615,1040.000000,1784.005884,5070.000000,4788.531160,29490.000000,31926.540458,2021-01-18
12,2021-01-30,22230.00000,11051.662112,29520.000000,21598.725667,6700.000000,11310.475941,52790.000000,29312.056758,15740.000000,...,5076.964364,940.000000,733.831878,1560.000000,720.592875,1090.000000,1581.194420,32540.000000,29388.639196,2021-01-18
13,2021-01-31,25230.00000,6645.548423,10610.000000,12271.505745,9020.000000,5279.232805,260.000000,904.163901,6450.000000,...,2479.524593,1050.000000,383.206522,760.000000,742.008143,1780.000000,1747.110826,6270.000000,7340.856359,2021-01-18
14,MAPE,0.33180,0.331800,0.217646,0.217646,0.329513,0.329513,0.589493,0.589493,0.408300,...,0.625813,2.287982,2.287982,0.809081,0.809081,0.597869,0.597869,0.181466,0.181466,2021-01-18


In [3]:
# Get a list of column names to keep
columns_to_keep = [column for column in xgb_raw_df.columns if not ends_with_number(column, numbers_to_remove)]

# Remove the columns from the data frame
xgb_raw_df = xgb_raw_df[columns_to_keep]

## MAPE averages for XGB before and after COVID

In [4]:
# Filter rows where date equals 'MAPE'
mape_df = xgb_raw_df[xgb_raw_df['date'] == 'MAPE']
mape_df = mape_df.drop('date', axis=1)

mape_df['mean_except_periodID'] = mape_df.drop('periodID', axis=1).mean(axis=1)
mape_df = mape_df.rename(columns={'mean_except_periodID': 'mape'})

mape_df = mape_df[['periodID', 'mape']]

mape_df_sorted = mape_df.sort_values('periodID')
mape_first_5_avg = mape_df_sorted.iloc[:5]['mape'].mean()
mape_last_5_avg = mape_df_sorted.iloc[-5:]['mape'].mean()

print(mape_first_5_avg)
print(mape_last_5_avg)


0.49681420670128784
0.6772665629924495


## SMAPE averages for XGB before and after COVID

In [5]:
# Filter rows where date equals 'MAPE'
mape_df = xgb_raw_df[xgb_raw_df['date'] == 'SMAPE']
mape_df = mape_df.drop('date', axis=1)

mape_df['mean_except_periodID'] = mape_df.drop('periodID', axis=1).mean(axis=1)
mape_df = mape_df.rename(columns={'mean_except_periodID': 'smape'})

mape_df = mape_df[['periodID', 'smape']]

mape_df_sorted = mape_df.sort_values('periodID')
smape_first_5_avg = mape_df_sorted.iloc[:5]['smape'].mean()
smape_last_5_avg = mape_df_sorted.iloc[-5:]['smape'].mean()

print(smape_first_5_avg)
print(smape_last_5_avg)

0.32204795297442756
0.39102227864539285


## MAPE & SMAPE averages for XGB

In [6]:
print(xgb_raw_df[xgb_raw_df['date'] == 'MAPE'].mean().mean())
print(xgb_raw_df[xgb_raw_df['date'] == 'SMAPE'].mean().mean())

0.5870403848468686
0.3565351158099102


C:\Users\amachno\AppData\Local\Temp\ipykernel_23520\499029310.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(xgb_raw_df[xgb_raw_df['date'] == 'MAPE'].mean().mean())
C:\Users\amachno\AppData\Local\Temp\ipykernel_23520\499029310.py:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(xgb_raw_df[xgb_raw_df['date'] == 'SMAPE'].mean().mean())


In [7]:
# Define the folder where the CSV files are located
folder_path = r'D:\AGH\bankomaty_2022\data\redatyprognoz'

# Use glob to find all CSV files in the folder
file_pattern = folder_path + r'\prognozy_rf_W_*.csv'
file_list = glob.glob(file_pattern)

# Create an empty list to hold the DataFrames
df_list = []

# Loop over each file and read it into a DataFrame
for file in file_list:
    df = pd.read_csv(file)
    period_id = df['date'].min()
    df['periodID'] = period_id
    df_list.append(df)

# Concatenate the DataFrames into a single DataFrame
rf_raw_df = pd.concat(df_list)

# Concatenate the DataFrames into a single DataFrame
rf_raw_df  = pd.concat(df_list)

rf_raw_df

,date,ATM_W_01,pred_W_01,ATM_W_02,pred_W_02,ATM_W_03,pred_W_03,ATM_W_04,pred_W_04,ATM_W_05,...,pred_W_77,ATM_W_78,pred_W_78,ATM_W_79,pred_W_79,ATM_W_80,pred_W_80,ATM_W_81,pred_W_81,periodID
0,2019-04-26,0.000000,20166.285690,36730.000000,32766.524901,24140.000000,20960.694883,53250.000000,37797.546587,55470.000000,...,13055.311122,21070.000000,23744.620918,5030.000000,3145.085888,14040.000000,11330.207121,50940.000000,34664.193547,2019-04-26
1,2019-04-27,0.000000,6096.599247,28830.000000,22295.105192,13960.000000,14375.656181,75880.000000,33144.924752,20020.000000,...,10607.545328,11790.000000,13944.390134,4700.000000,2536.355049,14810.000000,7194.329813,65700.000000,33308.446461,2019-04-26
2,2019-04-28,0.000000,1321.242284,9740.000000,12463.139913,4810.000000,6436.158510,0.000000,1053.229604,10570.000000,...,3337.574592,11930.000000,2573.745901,790.000000,1798.295502,6110.000000,5156.888963,12430.000000,6547.842507,2019-04-26
3,2019-04-29,0.000000,7767.361564,56280.000000,31510.638859,22570.000000,17327.327230,32950.000000,24876.718810,39740.000000,...,15820.120142,34590.000000,21888.901549,2290.000000,3535.043665,28830.000000,10214.680037,39220.000000,34063.701660,2019-04-26
4,2019-04-30,0.000000,6095.052924,63850.000000,40598.267807,27850.000000,24228.939612,62430.000000,42543.812821,38090.000000,...,19835.398281,30310.000000,30776.951583,3980.000000,3187.017393,20320.000000,16140.474454,52650.000000,44278.482831,2019-04-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,2021-01-29,14760.000000,19279.280869,28910.000000,35496.888183,36180.000000,16363.141028,25520.000000,33444.527849,34020.000000,...,11486.787131,720.000000,3885.164287,1040.000000,1435.925629,5070.000000,4019.263970,29490.000000,26987.775650,2021-01-18
12,2021-01-30,22230.000000,11197.924476,29520.000000,23111.020560,6700.000000,12912.711859,52790.000000,30072.460117,15740.000000,...,5974.590660,940.000000,1095.406446,1560.000000,800.001581,1090.000000,2245.810737,32540.000000,27143.053038,2021-01-18
13,2021-01-31,25230.000000,6444.721224,10610.000000,11678.203028,9020.000000,5180.396296,260.000000,1623.824890,6450.000000,...,2381.205228,1050.000000,795.010521,760.000000,687.273542,1780.000000,1497.825855,6270.000000,7002.801772,2021-01-18
14,MAPE,0.341337,0.341337,0.171827,0.171827,0.323655,0.323655,0.805237,0.805237,0.424947,...,0.654784,2.177730,2.177730,0.844588,0.844588,0.538018,0.538018,0.162649,0.162649,2021-01-18


In [8]:
# Get a list of column names to keep
columns_to_keep = [column for column in rf_raw_df.columns if not ends_with_number(column, numbers_to_remove)]

# Remove the columns from the data frame
rf_raw_df = rf_raw_df[columns_to_keep]

## MAPE averages for RF before and after COVID

In [9]:
# Filter rows where date equals 'MAPE'
mape_df = rf_raw_df[rf_raw_df['date'] == 'MAPE']
mape_df = mape_df.drop('date', axis=1)

mape_df['mean_except_periodID'] = mape_df.drop('periodID', axis=1).mean(axis=1)
mape_df = mape_df.rename(columns={'mean_except_periodID': 'mape'})

mape_df = mape_df[['periodID', 'mape']]

mape_df_sorted = mape_df.sort_values('periodID')
mape_first_5_avg = mape_df_sorted.iloc[:5]['mape'].mean()
mape_last_5_avg = mape_df_sorted.iloc[-5:]['mape'].mean()

print(mape_first_5_avg)
print(mape_last_5_avg)


0.5083461744278248
0.700496127174886


## SMAPE averages for RF before and after COVID

In [10]:
# Filter rows where date equals 'MAPE'
mape_df = rf_raw_df[rf_raw_df['date'] == 'SMAPE']
mape_df = mape_df.drop('date', axis=1)

mape_df['mean_except_periodID'] = mape_df.drop('periodID', axis=1).mean(axis=1)
mape_df = mape_df.rename(columns={'mean_except_periodID': 'smape'})

mape_df = mape_df[['periodID', 'smape']]

mape_df_sorted = mape_df.sort_values('periodID')
smape_first_5_avg = mape_df_sorted.iloc[:5]['smape'].mean()
smape_last_5_avg = mape_df_sorted.iloc[-5:]['smape'].mean()

print(smape_first_5_avg)
print(smape_last_5_avg)

0.3206192809515783
0.3867038267476305


## MAPE & SMAPE averages for RF

In [11]:
print(rf_raw_df[rf_raw_df['date'] == 'MAPE'].mean().mean())
print(rf_raw_df[rf_raw_df['date'] == 'SMAPE'].mean().mean())

0.6044211508013553
0.35366155384960435


C:\Users\amachno\AppData\Local\Temp\ipykernel_23520\1468246873.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(rf_raw_df[rf_raw_df['date'] == 'MAPE'].mean().mean())
C:\Users\amachno\AppData\Local\Temp\ipykernel_23520\1468246873.py:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(rf_raw_df[rf_raw_df['date'] == 'SMAPE'].mean().mean())


# Import SARIMA

In [12]:
import os


# Define the directory where the Excel files are stored
directory = "D:/AGH/bankomaty_2022/data/Wielkość wypłat"

# Get a list of all Excel files in the directory
excel_files = [f for f in os.listdir(directory) if f.endswith(".xlsx")]

# Create an empty list to store the data frames
data_frames = []

# Loop through each Excel file and load the "prediction errors" sheet
for i, excel_file in enumerate(excel_files):
    file_path = os.path.join(directory, excel_file)
    xl = pd.ExcelFile(file_path)
    sheet_name = "prediction errors"
    df = xl.parse(sheet_name)
    period_id = os.path.splitext(excel_file)[0]
    df['periodID'] = period_id
    data_frames.append(df)

# Combine all data frames into a single data frame
sarima_raw_df = pd.concat(data_frames)

sarima_raw_df

,name,ATM_W_01,ATM_W_02,ATM_W_03,ATM_W_04,ATM_W_05,ATM_W_06,ATM_W_07,ATM_W_08,ATM_W_09,...,ATM_W_73,ATM_W_74,ATM_W_75,ATM_W_76,ATM_W_77,ATM_W_78,ATM_W_79,ATM_W_80,ATM_W_81,periodID
0,ME,-2.230876e+03,-3.082905e+02,1.813385e+03,2.214465e+04,-2.552164e+03,2.724609e+03,1.473101e+04,2.804433e+03,1.652576e+03,...,5.714414e+03,1.200465e+04,5.640735e+03,4.037508e+03,2.200565e+03,5.393615e+03,1.075205e+03,-8.767979e+02,NaN,arima_2018-04-26_2019-04-25_n-14
1,MAE,1.116116e+04,7.992481e+03,4.618839e+03,2.360757e+04,9.148118e+03,1.591592e+04,1.847641e+04,1.442495e+04,2.607552e+03,...,8.943712e+03,1.203546e+04,1.281186e+04,6.850870e+03,6.874953e+03,8.330978e+03,1.516348e+03,7.288534e+03,NaN,arima_2018-04-26_2019-04-25_n-14
2,MSE,1.641409e+08,1.224581e+08,3.988579e+07,1.284793e+09,1.343562e+08,3.411035e+08,1.055145e+09,2.965179e+08,1.329307e+07,...,1.439595e+08,2.112034e+08,2.634327e+08,9.478960e+07,6.829901e+07,2.575235e+08,4.969216e+06,8.907663e+07,NaN,arima_2018-04-26_2019-04-25_n-14
3,SDE,8.347174e+03,1.153517e+03,6.785065e+03,6.510249e+04,9.549323e+03,1.019455e+04,5.511838e+04,1.049323e+04,6.183374e+03,...,2.138138e+04,4.491727e+04,2.110570e+04,1.510697e+04,8.233759e+03,1.873956e+04,4.023049e+03,3.280677e+03,0.0,arima_2018-04-26_2019-04-25_n-14
4,MPE,-4.822076e+01,-8.812095e+00,-3.679707e+01,2.846038e+01,-4.069147e+01,-4.748977e+01,1.463793e+01,-1.785822e+01,1.109492e+01,...,1.051662e+01,4.502001e+01,-2.607530e+01,-1.269989e+01,-1.015190e+02,-2.710621e+00,1.421099e+01,-8.389696e+01,NaN,arima_2018-04-26_2019-04-25_n-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,MSE,6.288333e+07,3.138319e+07,6.785570e+07,NaN,1.118167e+08,1.152425e+08,6.248732e+07,1.694552e+08,1.755161e+07,...,5.727571e+07,1.742768e+07,8.969394e+07,5.206614e+07,2.800187e+07,1.519844e+08,1.401830e+06,NaN,NaN,arima_2020-01-18_2021-01-31_n-14
3,SDE,8.166631e+03,4.241822e+02,1.565801e+04,0.000000e+00,6.321283e+03,3.136939e+04,4.648496e+03,1.668496e+04,1.448115e+03,...,2.343402e+04,8.452906e+03,8.365224e+03,1.277475e+04,2.725579e+03,3.651159e+04,1.393992e+01,0.000000e+00,0.0,arima_2020-01-18_2021-01-31_n-14
4,MPE,-2.015716e+01,-3.112332e-01,1.670046e+01,NaN,-2.490947e+01,3.275226e+01,-1.878680e+00,2.725439e+00,-1.630216e+01,...,3.038828e+01,3.337917e+01,4.908063e+00,-1.112765e+02,-3.198104e+01,-7.540816e+02,-6.927847e+01,NaN,NaN,arima_2020-01-18_2021-01-31_n-14
5,MAPE,3.708332e+01,1.682892e+01,2.840645e+01,NaN,4.093625e+01,3.275226e+01,2.283591e+01,2.405784e+01,3.093892e+01,...,3.116348e+01,4.960397e+01,1.801190e+01,1.284564e+02,6.376062e+01,7.540816e+02,1.017955e+02,NaN,NaN,arima_2020-01-18_2021-01-31_n-14


In [13]:
# Get a list of column names to keep
columns_to_keep = [column for column in sarima_raw_df.columns if not ends_with_number(column, numbers_to_remove)]

# Remove the columns from the data frame
sarima_raw_df = sarima_raw_df[columns_to_keep]

## MAPE averages for SARIMA before and after COVID

In [14]:
# Filter rows where date equals 'MAPE'
mape_df = sarima_raw_df[sarima_raw_df['name'] == 'MAPE']
mape_df = mape_df.drop('name', axis=1)

mape_df['mean_except_periodID'] = mape_df.drop('periodID', axis=1).mean(axis=1)
mape_df = mape_df.rename(columns={'mean_except_periodID': 'mape'})

mape_df = mape_df[['periodID', 'mape']]

mape_df_sorted = mape_df.sort_values('periodID')
mape_first_5_avg = mape_df_sorted.iloc[:5]['mape'].mean()
mape_last_5_avg = mape_df_sorted.iloc[-5:]['mape'].mean()

print(mape_first_5_avg)
print(mape_last_5_avg)

57.58324077762184
65.0415783591531


## SMAPE averages for SARIMA before and after COVID

In [15]:
# Filter rows where date equals 'MAPE'
mape_df = sarima_raw_df[sarima_raw_df['name'] == 'SMAPE']
mape_df = mape_df.drop('name', axis=1)

mape_df['mean_except_periodID'] = mape_df.drop('periodID', axis=1).mean(axis=1)
mape_df = mape_df.rename(columns={'mean_except_periodID': 'mape'})

mape_df = mape_df[['periodID', 'mape']]

mape_df_sorted = mape_df.sort_values('periodID')
smape_first_5_avg = mape_df_sorted.iloc[:5]['mape'].mean()
smape_last_5_avg = mape_df_sorted.iloc[-5:]['mape'].mean()

print(smape_first_5_avg)
print(smape_last_5_avg)

34.94974042324493
47.083234524147144


## MAPE & SMAPE averages for SARIMA

In [16]:
print(sarima_raw_df[sarima_raw_df['name'] == 'MAPE'].mean().mean())
print(sarima_raw_df[sarima_raw_df['name'] == 'SMAPE'].mean().mean())

61.965949450889816
41.33513497653343


C:\Users\amachno\AppData\Local\Temp\ipykernel_23520\2902914132.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(sarima_raw_df[sarima_raw_df['name'] == 'MAPE'].mean().mean())
C:\Users\amachno\AppData\Local\Temp\ipykernel_23520\2902914132.py:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(sarima_raw_df[sarima_raw_df['name'] == 'SMAPE'].mean().mean())


# Import BAYES

In [17]:
import pandas as pd

# Define the file path of the Excel file
file_path = r'D:\AGH\bankomaty_2022\data\replaced 17-12-2022.xlsx'

# Read all sheets with names ending in "_errors" into a list of data frames
df_list = []
xl = pd.ExcelFile(file_path)
for sheet_name in xl.sheet_names:
    if sheet_name.endswith("_errors"):
        df = xl.parse(sheet_name)
        period_id = sheet_name[:-7]
        day, month, year = period_id.split("_")
        new_period_id = f"{year}_{month}_{day}"
        df['periodID'] = new_period_id
        df_list.append(df)

# Concatenate the data frames into a single data frame
bayes_raw_df = pd.concat(df_list)

# Rename the "Unnamed" column to "name"
bayes_raw_df = bayes_raw_df.rename(columns={"Unnamed: 0": "name"})

# Remove columns with names starting with 'ATM_N_'
bayes_raw_df = bayes_raw_df.loc[:, ~bayes_raw_df.columns.str.startswith('ATM_N_')]

# Print the resulting data frame
bayes_raw_df


,name,ATM_W_01,ATM_W_02,ATM_W_03,ATM_W_04,ATM_W_05,ATM_W_06,ATM_W_07,ATM_W_08,ATM_W_09,...,ATM_W_73,ATM_W_74,ATM_W_75,ATM_W_76,ATM_W_77,ATM_W_78,ATM_W_79,ATM_W_80,ATM_W_81,periodID
0,ME,-5.491916e+03,-3.496988e+02,-4.497321e+03,-1.214664e+04,-1.880867e+03,-7.977589e+03,-3.756119e+03,-1.318322e+04,-2.587950e+03,...,-4.089955e+03,-4.847893e+03,-3.368521e+03,4.858671e+03,-5.767666e+03,-2.785586e+03,-8.564753e+02,3.856033e+03,NaN,2019_01_18
1,MAE,1.001571e+04,1.029106e+04,5.397811e+03,1.441558e+04,9.144990e+03,1.128075e+04,1.227990e+04,1.589335e+04,3.810708e+03,...,7.537487e+03,9.269531e+03,1.148109e+04,1.124215e+04,6.661174e+03,7.824904e+03,1.415952e+03,3.856033e+03,NaN,2019_01_18
2,MSE,1.566379e+08,1.801695e+08,3.796089e+07,2.804660e+08,1.394666e+08,1.487286e+08,2.264183e+08,4.591269e+08,2.038533e+07,...,8.679401e+07,1.191241e+08,1.792140e+08,2.235983e+08,7.723942e+07,8.678772e+07,2.407666e+06,2.787238e+07,NaN,2019_01_18
3,SDE,2.054887e+04,1.308453e+03,1.682743e+04,3.895592e+04,7.037559e+03,2.984940e+04,1.405411e+04,4.932711e+04,9.683224e+03,...,1.530321e+04,1.813916e+04,1.260385e+04,1.688095e+04,2.158063e+04,1.042271e+04,3.204637e+03,2.061136e+03,0.000000e+00,2019_01_18
4,MPE,-5.012392e+01,-2.316828e+01,-5.378787e+01,-8.818766e+01,-2.148561e+01,-3.577531e+01,-2.378845e+01,-4.871296e+01,-5.218428e+01,...,-3.903770e+01,-8.581162e+01,-2.186079e+01,-1.029155e+01,-3.421632e+02,-2.590917e+02,-1.025708e+02,3.088564e+01,NaN,2019_01_18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,MSE,9.414965e+07,9.084370e+07,3.063200e+07,1.013653e+09,2.591101e+08,1.503316e+08,8.404502e+07,4.325103e+08,4.826819e+07,...,1.121807e+08,2.115769e+07,2.490900e+08,9.179606e+07,2.559371e+07,3.953848e+07,3.732145e+07,3.852969e+06,9.691503e+08,2020_12_15
3,SDE,3.311533e+03,4.302961e+03,1.681818e+03,5.317509e+04,1.620752e+04,4.114831e+03,1.487792e+04,3.408063e+04,3.631043e+03,...,2.093452e+04,1.052675e+04,2.567423e+04,9.661550e+03,9.918275e+03,1.195846e+04,8.096520e+03,5.456205e+03,6.476375e+04,2020_12_15
4,MPE,-1.743592e+01,-8.563208e+00,-6.199399e+01,3.161425e+01,-5.153497e+01,-5.601522e+01,5.544603e+00,2.672958e+00,-7.155327e+01,...,-6.930373e+01,-2.683963e+02,-3.576564e+01,-2.498910e+02,2.174741e+01,1.672637e+01,-9.515155e+01,-1.120822e+02,8.035608e+01,2020_12_15
5,MAPE,3.239350e+01,3.153551e+01,8.265560e+01,3.723825e+01,8.881617e+01,8.017197e+01,2.580421e+01,3.004507e+01,1.045662e+02,...,7.770712e+01,2.816474e+02,6.984840e+01,2.675619e+02,4.028502e+01,3.386075e+01,1.488566e+02,1.157882e+02,8.035608e+01,2020_12_15


In [18]:
# Get a list of column names to keep
columns_to_keep = [column for column in bayes_raw_df.columns if not ends_with_number(column, numbers_to_remove)]

# Remove the columns from the data frame
bayes_raw_df = bayes_raw_df[columns_to_keep]

In [19]:
bayes_raw_df

,name,ATM_W_01,ATM_W_02,ATM_W_03,ATM_W_04,ATM_W_05,ATM_W_06,ATM_W_07,ATM_W_08,ATM_W_09,...,ATM_W_70,ATM_W_71,ATM_W_72,ATM_W_73,ATM_W_74,ATM_W_75,ATM_W_76,ATM_W_77,ATM_W_78,periodID
0,ME,-5.491916e+03,-3.496988e+02,-4.497321e+03,-1.214664e+04,-1.880867e+03,-7.977589e+03,-3.756119e+03,-1.318322e+04,-2.587950e+03,...,-5.511644e+03,-3.052901e+03,2.794585e+03,-4.089955e+03,-4.847893e+03,-3.368521e+03,4.858671e+03,-5.767666e+03,-2.785586e+03,2019_01_18
1,MAE,1.001571e+04,1.029106e+04,5.397811e+03,1.441558e+04,9.144990e+03,1.128075e+04,1.227990e+04,1.589335e+04,3.810708e+03,...,6.914901e+03,8.141837e+03,8.457523e+03,7.537487e+03,9.269531e+03,1.148109e+04,1.124215e+04,6.661174e+03,7.824904e+03,2019_01_18
2,MSE,1.566379e+08,1.801695e+08,3.796089e+07,2.804660e+08,1.394666e+08,1.487286e+08,2.264183e+08,4.591269e+08,2.038533e+07,...,6.976873e+07,1.293371e+08,1.366304e+08,8.679401e+07,1.191241e+08,1.792140e+08,2.235983e+08,7.723942e+07,8.678772e+07,2019_01_18
3,SDE,2.054887e+04,1.308453e+03,1.682743e+04,3.895592e+04,7.037559e+03,2.984940e+04,1.405411e+04,4.932711e+04,9.683224e+03,...,2.062268e+04,1.142291e+04,1.045638e+04,1.530321e+04,1.813916e+04,1.260385e+04,1.688095e+04,2.158063e+04,1.042271e+04,2019_01_18
4,MPE,-5.012392e+01,-2.316828e+01,-5.378787e+01,-8.818766e+01,-2.148561e+01,-3.577531e+01,-2.378845e+01,-4.871296e+01,-5.218428e+01,...,-4.157751e+01,-1.636435e+02,-1.535896e+01,-3.903770e+01,-8.581162e+01,-2.186079e+01,-1.029155e+01,-3.421632e+02,-2.590917e+02,2019_01_18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,MSE,9.414965e+07,9.084370e+07,3.063200e+07,1.013653e+09,2.591101e+08,1.503316e+08,8.404502e+07,4.325103e+08,4.826819e+07,...,1.240976e+08,2.484911e+06,4.987024e+07,1.121807e+08,2.115769e+07,2.490900e+08,9.179606e+07,2.559371e+07,3.953848e+07,2020_12_15
3,SDE,3.311533e+03,4.302961e+03,1.681818e+03,5.317509e+04,1.620752e+04,4.114831e+03,1.487792e+04,3.408063e+04,3.631043e+03,...,1.364438e+04,2.648195e+03,2.279629e+03,2.093452e+04,1.052675e+04,2.567423e+04,9.661550e+03,9.918275e+03,1.195846e+04,2020_12_15
4,MPE,-1.743592e+01,-8.563208e+00,-6.199399e+01,3.161425e+01,-5.153497e+01,-5.601522e+01,5.544603e+00,2.672958e+00,-7.155327e+01,...,-2.453166e+01,-1.840322e+02,-6.878473e+01,-6.930373e+01,-2.683963e+02,-3.576564e+01,-2.498910e+02,2.174741e+01,1.672637e+01,2020_12_15
5,MAPE,3.239350e+01,3.153551e+01,8.265560e+01,3.723825e+01,8.881617e+01,8.017197e+01,2.580421e+01,3.004507e+01,1.045662e+02,...,6.170645e+01,2.095121e+02,9.341079e+01,7.770712e+01,2.816474e+02,6.984840e+01,2.675619e+02,4.028502e+01,3.386075e+01,2020_12_15


## MAPE averages for BAYES

In [20]:
# Filter rows where date equals 'MAPE'
mape_df = bayes_raw_df[bayes_raw_df['name'] == 'MAPE']
mape_df = mape_df.drop('name', axis=1)

mape_df['mean_except_periodID'] = mape_df.drop('periodID', axis=1).mean(axis=1)
mape_df = mape_df.rename(columns={'mean_except_periodID': 'mape'})

mape_df = mape_df[['periodID', 'mape']]

mape_df_sorted = mape_df.sort_values('periodID')
mape_first_5_avg = mape_df_sorted.iloc[:5]['mape'].mean()
mape_last_5_avg = mape_df_sorted.iloc[-5:]['mape'].mean()

print(mape_first_5_avg)
print(mape_last_5_avg)

89.7576668811974
370.10459759227626


## SMAPE averages for BAYES

In [21]:
# Filter rows where date equals 'MAPE'
mape_df = bayes_raw_df[bayes_raw_df['name'] == 'SMAPE']
mape_df = mape_df.drop('name', axis=1)

mape_df['mean_except_periodID'] = mape_df.drop('periodID', axis=1).mean(axis=1)
mape_df = mape_df.rename(columns={'mean_except_periodID': 'mape'})

mape_df = mape_df[['periodID', 'mape']]

mape_df_sorted = mape_df.sort_values('periodID')
smape_first_5_avg = mape_df_sorted.iloc[:5]['mape'].mean()
smape_last_5_avg = mape_df_sorted.iloc[-5:]['mape'].mean()

print(smape_first_5_avg)
print(smape_last_5_avg)

57.0122367321105
53.29531564981448


In [22]:
bayes_raw_df

,name,ATM_W_01,ATM_W_02,ATM_W_03,ATM_W_04,ATM_W_05,ATM_W_06,ATM_W_07,ATM_W_08,ATM_W_09,...,ATM_W_70,ATM_W_71,ATM_W_72,ATM_W_73,ATM_W_74,ATM_W_75,ATM_W_76,ATM_W_77,ATM_W_78,periodID
0,ME,-5.491916e+03,-3.496988e+02,-4.497321e+03,-1.214664e+04,-1.880867e+03,-7.977589e+03,-3.756119e+03,-1.318322e+04,-2.587950e+03,...,-5.511644e+03,-3.052901e+03,2.794585e+03,-4.089955e+03,-4.847893e+03,-3.368521e+03,4.858671e+03,-5.767666e+03,-2.785586e+03,2019_01_18
1,MAE,1.001571e+04,1.029106e+04,5.397811e+03,1.441558e+04,9.144990e+03,1.128075e+04,1.227990e+04,1.589335e+04,3.810708e+03,...,6.914901e+03,8.141837e+03,8.457523e+03,7.537487e+03,9.269531e+03,1.148109e+04,1.124215e+04,6.661174e+03,7.824904e+03,2019_01_18
2,MSE,1.566379e+08,1.801695e+08,3.796089e+07,2.804660e+08,1.394666e+08,1.487286e+08,2.264183e+08,4.591269e+08,2.038533e+07,...,6.976873e+07,1.293371e+08,1.366304e+08,8.679401e+07,1.191241e+08,1.792140e+08,2.235983e+08,7.723942e+07,8.678772e+07,2019_01_18
3,SDE,2.054887e+04,1.308453e+03,1.682743e+04,3.895592e+04,7.037559e+03,2.984940e+04,1.405411e+04,4.932711e+04,9.683224e+03,...,2.062268e+04,1.142291e+04,1.045638e+04,1.530321e+04,1.813916e+04,1.260385e+04,1.688095e+04,2.158063e+04,1.042271e+04,2019_01_18
4,MPE,-5.012392e+01,-2.316828e+01,-5.378787e+01,-8.818766e+01,-2.148561e+01,-3.577531e+01,-2.378845e+01,-4.871296e+01,-5.218428e+01,...,-4.157751e+01,-1.636435e+02,-1.535896e+01,-3.903770e+01,-8.581162e+01,-2.186079e+01,-1.029155e+01,-3.421632e+02,-2.590917e+02,2019_01_18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,MSE,9.414965e+07,9.084370e+07,3.063200e+07,1.013653e+09,2.591101e+08,1.503316e+08,8.404502e+07,4.325103e+08,4.826819e+07,...,1.240976e+08,2.484911e+06,4.987024e+07,1.121807e+08,2.115769e+07,2.490900e+08,9.179606e+07,2.559371e+07,3.953848e+07,2020_12_15
3,SDE,3.311533e+03,4.302961e+03,1.681818e+03,5.317509e+04,1.620752e+04,4.114831e+03,1.487792e+04,3.408063e+04,3.631043e+03,...,1.364438e+04,2.648195e+03,2.279629e+03,2.093452e+04,1.052675e+04,2.567423e+04,9.661550e+03,9.918275e+03,1.195846e+04,2020_12_15
4,MPE,-1.743592e+01,-8.563208e+00,-6.199399e+01,3.161425e+01,-5.153497e+01,-5.601522e+01,5.544603e+00,2.672958e+00,-7.155327e+01,...,-2.453166e+01,-1.840322e+02,-6.878473e+01,-6.930373e+01,-2.683963e+02,-3.576564e+01,-2.498910e+02,2.174741e+01,1.672637e+01,2020_12_15
5,MAPE,3.239350e+01,3.153551e+01,8.265560e+01,3.723825e+01,8.881617e+01,8.017197e+01,2.580421e+01,3.004507e+01,1.045662e+02,...,6.170645e+01,2.095121e+02,9.341079e+01,7.770712e+01,2.816474e+02,6.984840e+01,2.675619e+02,4.028502e+01,3.386075e+01,2020_12_15


## MAPE & SMAPE averages for BAYES

In [23]:
print(bayes_raw_df.drop('periodID', axis=1).mean(axis=1)[bayes_raw_df['name'] == 'MAPE'].mean().mean())
print(bayes_raw_df.drop('periodID', axis=1).mean(axis=1)[bayes_raw_df['name'] == 'SMAPE'].mean().mean())

244.35844634899792
56.13340584402275


C:\Users\amachno\AppData\Local\Temp\ipykernel_23520\3637169021.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print(bayes_raw_df.drop('periodID', axis=1).mean(axis=1)[bayes_raw_df['name'] == 'MAPE'].mean().mean())
C:\Users\amachno\AppData\Local\Temp\ipykernel_23520\3637169021.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print(bayes_raw_df.drop('periodID', axis=1).mean(axis=1)[bayes_raw_df['name'] == 'SMAPE'].mean().mean())


# Rankings per period - MAPE

In [24]:
from utils import change_id 
from utils import filter_and_remove
import pyperclip

measure = 'MAPE'

xgb_df = xgb_raw_df.rename(columns={'date': 'name'})
rf_df = rf_raw_df.rename(columns={'date': 'name'})

xgb_df = xgb_df.drop(xgb_df.filter(regex='^pred_W').columns, axis=1)
rf_df = rf_df.drop(rf_df.filter(regex='^pred_W').columns, axis=1)

xgb_df = change_id(xgb_df)
rf_df = change_id(rf_df)
sarima_df = change_id(sarima_raw_df)
bayes_df = change_id(bayes_raw_df)


xgb_df = filter_and_remove(xgb_df, measure)
rf_df = filter_and_remove(rf_df, measure)
sarima_df = filter_and_remove(sarima_df, measure)
bayes_df = filter_and_remove(bayes_df, measure)

xgb_df['model'] = 'xgb'
rf_df['model'] = 'rf'
sarima_df['model'] = 'sarima'
bayes_df['model'] = 'bayes'

xgb_df.loc[:, xgb_df.columns.str.startswith('ATM_W')] *= 100
rf_df.loc[:, rf_df.columns.str.startswith('ATM_W')] *= 100


### Temporary fix

In [25]:
# define a dictionary to map old values to new values
period_dict = {1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7:8, 8: 9, 9: 10}

# replace the old values with the new values in the "periodID" column
bayes_df['periodID'] = bayes_df['periodID'].map(period_dict).astype(int)


In [26]:
df_mape = pd.concat([xgb_df, rf_df, sarima_df, bayes_df], ignore_index=True)
df_mape = df_mape.melt(id_vars=['periodID', 'model'], var_name='ATM', value_name=measure)
df_mape['rank'] = df_mape.groupby(['periodID', 'ATM'])[measure].rank(method='min')


## MAPE ranking total

In [27]:

# count how many times each model has each ranking
counts = df_mape.groupby(['model', 'rank']).size().reset_index(name='count')

# pivot the counts dataframe to a wider format
pivoted_counts = counts.pivot(index='model', columns='rank', values='count')
pivoted_counts.columns = [f'rank_{col}' for col in pivoted_counts.columns]
pivoted_counts


,rank_1.0,rank_2.0,rank_3.0,rank_4.0
model,,,,
bayes,117,39,90,456
rf,167,357,212,44
sarima,265,85,252,142
xgb,231,299,224,26


## MAPE ranking before COVID

In [28]:
# filter the dataframe for periodID <= 5
df_mape_filtered = df_mape[df_mape['periodID'] <= 5]

# group and count the data
counts = df_mape_filtered.groupby(['model', 'rank']).size().reset_index(name='count')

# pivot the counts dataframe to a wider format
pivoted_counts = counts.pivot(index='model', columns='rank', values='count')
pivoted_counts.columns = [f'rank_{col}' for col in pivoted_counts.columns]
pivoted_counts

,rank_1.0,rank_2.0,rank_3.0,rank_4.0
model,,,,
bayes,68,26,47,249
rf,94,156,115,25
sarima,130,41,120,90
xgb,98,167,108,17


## MAPE ranking after COVID

In [29]:
# filter the dataframe for periodID > 5
df_mape_filtered = df_mape[df_mape['periodID'] > 5]

# group and count the data
counts = df_mape_filtered.groupby(['model', 'rank']).size().reset_index(name='count')

# pivot the counts dataframe to a wider format
pivoted_counts = counts.pivot(index='model', columns='rank', values='count')
pivoted_counts.columns = [f'rank_{col}' for col in pivoted_counts.columns]
pivoted_counts

,rank_1.0,rank_2.0,rank_3.0,rank_4.0
model,,,,
bayes,49,13,43,207
rf,73,201,97,19
sarima,135,44,132,52
xgb,133,132,116,9


## MAPE ranking without RF

In [30]:
# filter the dataframe for periodID > 5
df_mape_filtered = pd.concat([xgb_df, rf_df, sarima_df, bayes_df], ignore_index=True)
df_mape_filtered = df_mape_filtered[df_mape_filtered['model'] != 'rf']
df_mape_filtered = df_mape_filtered.melt(id_vars=['periodID', 'model'], var_name='ATM', value_name=measure)
df_mape_filtered['rank'] = df_mape_filtered.groupby(['periodID', 'ATM'])[measure].rank(method='min')
# group and count the data
counts = df_mape_filtered.groupby(['model', 'rank']).size().reset_index(name='count')

# pivot the counts dataframe to a wider format
pivoted_counts = counts.pivot(index='model', columns='rank', values='count')
pivoted_counts.columns = [f'rank_{col}' for col in pivoted_counts.columns]
pivoted_counts

,rank_1.0,rank_2.0,rank_3.0
model,,,
bayes,125,115,462
sarima,292,296,156
xgb,363,367,50
